# DSB Session 3 Demo - Unsupervised Learning Methods

In this demo we consider unsupervised learning in the form of clustering methods and principal component analysis (PCA).

This demo uses data from the Landsat dataset. 

## Notebook Setup

In [ ]:
import os
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))
import warnings
warnings.filterwarnings('ignore')

## Loading and exploring the landsat dataset

For information about the Landsat data you can read here: http://archive.ics.uci.edu/dataset/146/statlog+landsat+satellite


In [ ]:
# Read the CSV

data_path = os.path.join(os.getcwd(), 'datasets', 'landsat.csv')
landsat = pd.read_csv(data_path, delimiter = ',')

In [ ]:
# Get a feel for the dataset

landsat.head()

In [ ]:
landsat.shape

In [ ]:
# Split the data into the features `X`, and the labels `y`

X = landsat.drop('class', axis=1)
y = landsat['class'].values

In [ ]:
print('Number of instances: {}, number of attributes: {}'.format(X.shape[0], X.shape[1]))

In [ ]:
landsat.describe()

In [ ]:

labeldict = {
    1:'red soil', 
    2:'cotton crop', 
    3:'grey soil',
    4:'damp grey soil', 
    5:'soil with vegetation stubble',
    6:'mixture class (all types present)',
    7:'very damp grey soil'
}

fig, ax = plt.subplots()
landsat['class'].astype('category').value_counts().plot(kind='bar', ax=ax)
labels = [int(ticklabel.get_text()) for ticklabel in ax.get_xticklabels()]
ax.set_xticklabels([labeldict[l] for l in labels])
plt.xlabel('Classes')
plt.ylabel('Count')
plt.title('Class distribution')
plt.show()

In [ ]:
# Plot a few datapoints.

def get_images(row):    
    # Get each of the 4 3x3 images contained in the row
    # Pixels are labeled 1 to 9 from topleft to bottom right
    # They are measured at 4 spectral bands
    
    img = [[]] * 4
    for ii in range(4):
        img[ii] = row.iloc[[4*p + ii for p in range(9)]].values.reshape((3,3)).astype(int)
    return img
        
for ii in range(1):
    fig, ax = plt.subplots(1,4)
    plt.suptitle('Row {}'.format(ii), fontsize=16)
    for jj, img in enumerate(get_images(landsat.iloc[ii,:])):
        ax[jj] = sns.heatmap(
            img, 
            annot=True, 
            fmt="d", 
            ax=ax[jj], 
            vmin=0, 
            vmax=255,
            cbar=False, 
            square=True, 
            cmap=plt.cm.gray
        )
        ax[jj].invert_yaxis()
        ax[jj].set_title('Band {}'.format(jj))
    plt.tight_layout()
    plt.subplots_adjust(top=1.4)

In [ ]:

for ii in range(4):
    fig, ax = plt.subplots(1,4)
    plt.suptitle('Row {}'.format(ii), fontsize=16)
    for jj, img in enumerate(get_images(landsat.iloc[ii,:])):
        ax[jj] = sns.heatmap(
            img, 
            annot=True, 
            fmt="d", 
            ax=ax[jj], 
            vmin=0, 
            vmax=255,
            cbar=False, 
            square=True, 
            cmap=plt.cm.gray
        )
        ax[jj].invert_yaxis()
        ax[jj].set_title('Band {}'.format(jj))
    plt.tight_layout()
    plt.subplots_adjust(top=1.4)

## Clustering the landsat dataset

Since there are 6 classes in the data, it would be interesting to try clustering with k=6 centres...


Initialise a [k-means clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) 


In [ ]:
?KMeans

In [ ]:
# set a random_state such that we can reproduce our results

kmeans = KMeans(n_clusters=6, random_state=1337)

**Be careful to fit `X` - only the features - not the class labels!**

In [ ]:
kmeans.fit(X)

In [ ]:
kmeans.labels_

### Evaluating the K-Means Clustering

Since we have the true class labels in this case, we can use another metric: the [adjusted rand index](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation). 

In [ ]:
adjusted_rand_score(y, kmeans.labels_)

The sklearn documentation gives a great introduction to k-means [here](http://scikit-learn.org/stable/modules/clustering.html#k-means). It describes what the algorithm is trying to minimise - the squared difference between datapoints and their closest cluster centre - a.k.a. the **inertia**. Lower inertia implies a better fit.

In [ ]:
kmeans.inertia_

Let's have a look at the counts of the labels within each cluster. If the clustering has worked well, and the labels are inticative of genuine difference in the data, we should expect each cluster to have one dominant label.


In [ ]:

cats = ['red soil','cotton crop','grey soil', 'damp grey soil', 
        'soil with vegetation stubble', 'mixture class (all types present)',
        'very damp grey soil']
labeldict = dict(zip(range(1,8), cats))
y_labels = pd.Categorical([labeldict[y_i] for y_i in y],
                          categories=[c for c in cats if c != 'mixture class (all types present)']) # ensures colours same

fig, ax = plt.subplots(figsize=(10,6))
ax = sns.countplot(
    x=kmeans.labels_, 
    hue=y_labels, 
    ax=ax
)
ax.legend(loc='center left', bbox_to_anchor=[1.1, 0.5], title='Class')
plt.xlabel('cluster id')
plt.show()

### Evaluating different cluster numbers

In [ ]:
k_values = []
inertia_values = []
silouette_values = []

for k in range(2,13):
    kmeans = KMeans(n_clusters=k, random_state=1337)
    kmeans.fit(X)
    k_values.append(k)
    inertia_values.append(kmeans.inertia_)
    silouette_values.append(silhouette_score(X, kmeans.labels_))
    print(f'# Clusters: {k}')
    print(f'Inertia value: {kmeans.inertia_}')
    print(f'Silouette Score: {silhouette_score(X, kmeans.labels_)}')
    print()
    

In [ ]:
plt.scatter(k_values, inertia_values)
plt.plot(k_values, inertia_values)
plt.xlabel('# Clusters')
plt.ylabel('Inertia Score')
plt.title('Inertia Score')
plt.show()

In [ ]:
plt.scatter(k_values[3:], inertia_values[3:])
plt.plot(k_values[3:], inertia_values[3:])
plt.xlabel('# Clusters')
plt.ylabel('Inertia Score')
plt.title('Inertia Score')
plt.show()

In [ ]:
plt.scatter(k_values, silouette_values)
plt.plot(k_values, silouette_values)
plt.xlabel('# Clusters')
plt.ylabel('Silouette Score')
plt.title('Silouette Score')
plt.show()

## Dimensionality reduction

The landsat data is 36 dimensional, so we cannot visualise it, with respect to class, on a nice two dimensional plot. Additionally, as dimensionality increases, euclidean distance [becomes less meaningful](https://en.wikipedia.org/wiki/Curse_of_dimensionality#Distance_functions)...

Perhaps if we found a lower dimensional subspace the data lies upon, we could more easily distinguish the datapoints...


We are going to project the data down to 2 dimensions and visualise it using PCA. 


In [ ]:
?PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
X_2d = pca.fit_transform(X)

In [ ]:
X_2d

In [ ]:
pca.components_

### Visualisations

Let's visualise the data! Does the data look like it will be confused by a k-means clustering in the same way now?



Without performing a clustering, it's hard to say whether the data is more or less seperable. However, we can see 6 definite clusters with some overlap in the top plot, but it's not all that clear. The kernel density plot allows us to see better where the density lies. These clusters are not spherical and there is overlap, so k-means will always fail. See in particular the 'red soil' data points vs. the 'soil with vegetation stubble points. As expected, we see the 'cotton crop' data points are very distinct from the rest. 

In [ ]:

colours = matplotlib.rcParams['axes.prop_cycle'].by_key()['color']

plt.figure(figsize=(12,8))
colours = matplotlib.rcParams['axes.prop_cycle'].by_key()['color']
lw = 2

sub_cats = [c for c in cats if c != 'mixture class (all types present)']
for color, target_name in zip(colours, sub_cats):
    plt.scatter(
        X_2d[y_labels == target_name, 0], 
        X_2d[y_labels == target_name, 1], 
        color=color, 
        alpha=.5, 
        lw=lw,
        label=target_name
    )
plt.axis('equal')
plt.legend(loc='center left', scatterpoints=3, bbox_to_anchor=[1.01, 0.5])
plt.title('Labelled data in PCA space')
plt.xlabel('PC1')
plt.ylabel('PC2')
top_plot = plt.gca()
plt.show()

In [ ]:

import matplotlib.lines as mlines
fig, ax = plt.subplots(figsize=(12,8))
colours = matplotlib.rcParams['axes.prop_cycle'].by_key()['color']
lines = []
for ii, cat in enumerate(sub_cats):
    idx = y_labels==cat
    g = sns.kdeplot(
        x=X_2d[idx, 0], 
        y=X_2d[idx, 1], 
        ax=ax, 
        n_levels=10,
        cmap=sns.light_palette(colours[ii], as_cmap=True)
    )
ax.set_xlim(top_plot.get_xlim())
ax.set_ylim(top_plot.get_ylim())
patches = [mlines.Line2D([], [], color=colours[ii], label=cat) for ii, cat in enumerate(sub_cats)]
plt.legend(patches, sub_cats, loc='center left', scatterpoints=3, bbox_to_anchor=[1.01, 0.5])
plt.title('Kernel density plots for each class')
plt.show()

## Extension - experimenting with different cluster numbers on different dimension numbers

There seems to be a sweet spot around 10 clusters. This provides some evidence that the classes are multimodal. You can see that in the plots.

In [ ]:

for ii in range(1,20,2):
    km = KMeans(n_clusters=ii, random_state=1337)  
    km.fit(X_2d)
    y_pred = km.labels_
    print('\nk-means with {} clusters results\nARI: {}, Inertia: {}'.\
          format(ii, adjusted_rand_score(y, y_pred), km.inertia_))


Increasing from 2 to 3 principal component dimensions provides a big benefit for kmeans. In fact, it's better than using the full data. 

In [ ]:

for ii in range(2, 6):
    X_pca = PCA(n_components = ii).fit_transform(X)
    km = KMeans(n_clusters=6, random_state=1337)  
    km.fit(X_pca)
    y_pred = km.labels_
    print('\n{}D PCA data, kmeans results\nARI: {}, Inertia: {}'.\
          format(ii, adjusted_rand_score(y, y_pred), km.inertia_))

## Credits

Lab prepared by Lawrence Murray and Chris Williams, November 2008; revised Athina Spiliopoulou Nov 2009; revised Sean Moran Nov 2011; revised Boris Mitrovic Oct 2013; revised and converted python by James Owers and Agamemnon Krasoulis Oct 2016